In [3]:
import psycopg2
import pandas as pd

In [29]:
conn.close()
conn = psycopg2.connect("host=127.0.0.1 dbname=fortune500 user=postgres password=postgres")
cur = conn.cursor()

In [20]:
def create_database():
    #connect to default database
    conn = psycopg2.connect("host=127.0.0.1 dbname=postgres user=postgres password=postgres")
    conn.set_session(autocommit = True)
    cur = conn.cursor()

    #create new database
    cur.execute("DROP DATABASE IF EXISTS fortune500")
    cur.execute("CREATE DATABASE fortune500")

    #close default connection
    conn.close()

    #conn to new database
    conn = psycopg2.connect("host=127.0.0.1 dbname=fortune500 user=postgres password=postgres")
    cur = conn.cursor()

    return conn, cur

In [4]:
f500 = pd.read_csv(r'C:\Users\martin.bergmann\python_scripts\Projects\Postgres\data\fortune500.csv')

In [97]:
f500.head()

,Year,Rank,Company,Revenue (in millions),Profit (in millions)
0,1955,1,General Motors,9823.5,806
1,1955,2,Exxon Mobil,5661.4,584.8
2,1955,3,U.S. Steel,3250.4,195.4
3,1955,4,General Electric,2959.1,212.6
4,1955,5,Esmark,2510.8,19.1


In [17]:
f500.dtypes


Year         int64
Rank         int64
Company     object
Revenue    float64
Profit     float64
dtype: object

In [9]:
f500['Revenue (in millions)'] = pd.to_numeric(f500['Revenue (in millions)'], errors='coerce')

Year                       int64
Rank                       int64
Company                   object
Revenue (in millions)    float64
Profit (in millions)      object
dtype: object


In [10]:
f500['Profit (in millions)'] = pd.to_numeric(f500['Profit (in millions)'], errors='coerce')
print(f500.dtypes)

Year                       int64
Rank                       int64
Company                   object
Revenue (in millions)    float64
Profit (in millions)     float64
dtype: object


In [16]:
f500.describe()

,Year,Rank,Revenue,Profit
count,29500.000000,29500.000000,2.750000e+04,2.869600e+04
mean,1983.728814,284.397966,4.743083e+09,2.461071e+08
std,16.616417,191.401503,1.278372e+10,1.286469e+09
min,1955.000000,1.000000,4.970000e+07,-9.869600e+10
25%,1969.000000,135.000000,4.046000e+08,9.600000e+06
50%,1984.000000,269.000000,1.223350e+09,4.400000e+07
75%,1999.000000,403.000000,4.200750e+09,1.964000e+08
max,2009.000000,1000.000000,3.511390e+11,3.950000e+10


In [12]:
f500['Revenue'] = f500['Revenue (in millions)'] * 1000000

In [13]:
f500['Profit'] = f500['Profit (in millions)'] * 1000000

In [15]:
f500.drop(['Revenue (in millions)', 'Profit (in millions)'], axis=1, inplace=True)

In [35]:
conn, cur = create_database()
conn.commit()

In [30]:
f500_table = ("""CREATE TABLE IF NOT EXISTS f500(
    Year INT,
    Rank INT,
    Company VARCHAR,
    Revenue NUMERIC,
    Profit NUMERIC
    )""")

cur.execute(f500_table)
conn.commit()

In [31]:
f500_table_insert = ("""INSERT INTO f500(
    Year,
    Rank,
    Company,
    Revenue,
    Profit)
    VALUES(%s, %s, %s, %s, %s)
    """)

In [32]:
for index, row in f500.iterrows():
    cur.execute(f500_table_insert, list(row))



In [33]:
conn.commit()

In [34]:
conn.close()